<a href="https://colab.research.google.com/github/ekaterina533/dataset/blob/main/Camera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
!pip install pytest xmltodict

In [40]:
%%writefile main.py
import logging
import os
from logging.handlers import RotatingFileHandler
import json
import xmltodict
import re
import smtplib
from email.message import EmailMessage

class Config:
    def __init__(
        self,
        log_level: str = "DEBUG",
        json_path: str = "data.json",
        plates_paths: list = ["plates.xml"],
        email_login: str = "test@example.com",
        email_password: str = "password",
        email_server: str = "smtp.example.com",
        mail_send_to: str = "admin@example.com",
    ):
        self.log_level = log_level
        self.json_path = json_path
        self.plates_paths = plates_paths
        self.email_login = email_login
        self.email_password = email_password
        self.email_server = email_server
        self.mail_send_to = mail_send_to

    @classmethod
    def read_conf(cls):
        if not os.path.exists("config.json"):
            with open("config.json", "w", encoding="utf-8") as f:
                json.dump(cls().__dict__, f, indent=4)
        with open("config.json", encoding="utf-8") as f:
            data = json.load(f)
            config = cls()
            for key, value in data.items():
                setattr(config, key, value)
            return config

config = Config.read_conf()

logging.basicConfig(
    handlers=[
        RotatingFileHandler(
            "app.log",
            maxBytes=5*1024*1024,
            backupCount=2,
            encoding="utf-8"
        ),
        logging.StreamHandler()
    ],
    level=config.log_level,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

class UserObject:
    def __init__(self, plate: str, name: str, id: str):
        self.plate = plate
        self.name = name
        self.id = id
        logging.debug(f"Создан пользователь: {self.plate}, {self.name}, {self.id}")

def read_data_from_xml(file_path: str) -> list[UserObject]:
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            xml_data = file.read()
            data_dict = xmltodict.parse(xml_data)
            users = []

            rows = data_dict["Workbook"]["Worksheet"]["Table"]["Row"]
            if not isinstance(rows, list):
                rows = [rows]

            for row in rows:
                try:
                    cell_data = row["Cell"]
                    plate = cell_data[0]["Data"]["#text"]
                    name = cell_data[2]["Data"]["#text"]
                    user_id = re.findall(r"\[(.+?)\]", name)[0]
                    users.append(UserObject(plate, name, user_id))
                except (KeyError, IndexError, TypeError) as e:
                    logging.warning(f"Ошибка обработки строки: {e}")
            return users
    except Exception as e:
        logging.error(f"Ошибка чтения XML: {e}", exc_info=True)
        return []

def read_data_from_json(file_path: str) -> dict:
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return json.load(file)
    except Exception as e:
        logging.error(f"Ошибка чтения JSON: {e}", exc_info=True)
        return {}

def send_email(subject: str, body: str):
    try:
        msg = EmailMessage()
        msg["Subject"] = subject
        msg["From"] = config.email_login
        msg["To"] = config.mail_send_to
        msg.set_content(body, subtype="html")

        with smtplib.SMTP(config.email_server, 25) as server:
            server.ehlo()
            # server.starttls()
            # server.login(config.email_login, config.email_password)
            server.send_message(msg)
            logging.info("Письмо успешно отправлено")
    except Exception as e:
        logging.error(f"Ошибка отправки письма: {e}", exc_info=True)

def main():
    json_data = read_data_from_json(config.json_path)
    if not json_data:
        logging.error("Не удалось загрузить данные из JSON")
        return

    discrepancies = {}
    for path in config.plates_paths:
        users = read_data_from_xml(path)
        discrepancies[path] = []

        for user in users:
            try:
                if json_data.get(user.id, {}).get("Статус") is False:
                    discrepancies[path].append(user.name)
            except Exception as e:
                logging.warning(f"Ошибка проверки статуса: {e}")

    if not any(discrepancies.values()):
        logging.info("Нет расхождений для отправки")
        return

    email_body = "<h2>Обнаружены уволенные сотрудники с доступом:</h2><ul>"
    for path, names in discrepancies.items():
        if names:
            email_body += f"<li><b>{path}:</b><br>" + "<br>".join(names) + "</li>"
    email_body += "</ul>"

    send_email(
        "Уволенные сотрудники с доступом к парковке",
        email_body
    )

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logging.critical(f"Критическая ошибка: {e}", exc_info=True)

Overwriting main.py


In [41]:
%%writefile test_main.py
import pytest
from unittest.mock import patch, mock_open, MagicMock
import main
import json
import xmltodict
from main import send_email

# Тестовые данные с корректной структурой XML
TEST_XML = """
<Workbook>
  <Worksheet>
    <Table>
      <Row>
        <Cell><Data>А123БВ</Data></Cell>
        <Cell><Data>Тест</Data></Cell>
        <Cell><Data>Иванов Иван [ID123]</Data></Cell>
      </Row>
      <Row>
        <Cell><Data>Б456ВГ</Data></Cell>
        <Cell><Data>Тест</Data></Cell>
        <Cell><Data>Петров Петр [ID456]</Data></Cell>
      </Row>
    </Table>
  </Worksheet>
</Workbook>
"""

TEST_JSON = {
    "ID123": {"Статус": False},
    "ID456": {"Статус": True}
}

@pytest.fixture
def mock_config():
    return main.Config(
        log_level="DEBUG",
        json_path="test.json",
        plates_paths=["test.xml"],
        email_login="test@example.com",
        mail_send_to="admin@example.com"
    )

def test_read_data_from_json(tmp_path, mock_config):
    test_file = tmp_path / "test.json"
    test_file.write_text(json.dumps(TEST_JSON))

    result = main.read_data_from_json(str(test_file))
    assert result == TEST_JSON

def test_read_data_from_xml(tmp_path, mock_config):
    test_file = tmp_path / "test.xml"
    test_file.write_text(TEST_XML)

    users = main.read_data_from_xml(str(test_file))
    assert len(users) == 2
    assert users[0].plate == "А123БВ"
    assert users[0].id == "ID123"
    assert users[1].id == "ID456"

@patch('main.send_email')
@patch('main.read_data_from_json')
@patch('main.read_data_from_xml')
def test_main(mock_xml, mock_json, mock_email, mock_config):
    mock_json.return_value = TEST_JSON
    mock_xml.return_value = [
        main.UserObject("А123БВ", "Иванов Иван [ID123]", "ID123"),
        main.UserObject("Б456ВГ", "Петров Петр [ID456]", "ID456")
    ]

    with patch('main.config', mock_config):
        main.main()

    mock_email.assert_called_once()
    assert "Иванов Иван" in mock_email.call_args[0][1]

@patch('main.send_email')
def test_no_discrepancies(mock_email, mock_config):
    test_data = {"ID123": {"Статус": True}}

    with patch('main.read_data_from_json', return_value=test_data):
        with patch('main.read_data_from_xml'):
            with patch('main.config', mock_config):
                main.main()
                mock_email.assert_not_called()

def test_invalid_xml(tmp_path, caplog):
    test_file = tmp_path / "invalid.xml"
    test_file.write_text("Не XML")

    users = main.read_data_from_xml(str(test_file))
    assert len(users) == 0
    assert "Ошибка чтения XML" in caplog.text

Overwriting test_main.py


In [42]:
!pytest test_main.py -v

============================= test session starts ==============================
platform linux -- Python 3.11.13, pytest-8.4.1, pluggy-1.6.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-4.9.0, typeguard-4.4.4, langsmith-0.4.8
collected 5 items                                                              

test_main.py::test_read_data_from_json PASSED                            [ 20%]
test_main.py::test_read_data_from_xml FAILED                             [ 40%]
test_main.py::test_main PASSED                                           [ 60%]
test_main.py::test_no_discrepancies PASSED                               [ 80%]
test_main.py::test_invalid_xml PASSED                                    [100%]

=================================== FAILURES ===================================
___________________________ test_read_data_from_xml ____________________________

tmp_path = PosixPath('/tmp/pytest-of-root/pytest-4/test_read_data_from_xml0')
mock_config = <mai